In [60]:
import pandas as pd
import re
from konlpy.tag import Okt

In [61]:
# 형태소 분석기 선언
okt = Okt()

# 🔥 부정 키워드 목록 
negative_keywords = ['재미없', '노잼', '불편', '불만', '짜증', '실망', '별로', '최악', '지루', '유치해', 
    '답답', '이상', '비추', '불쾌', '싫', '아쉽', '꺼려', '나오고싶', '중간에 보다가', '그냥 그렇','속았음', 
    '아니', '할말없', '못', '아닌', '망작', '실패', '미흡', '부족', '어이없', '흥미가 계속되지않는다','하아',
    '집중은 힘들 것 같다', '안됨', '으악','ㅡㅡ','안본다','왜봤지','이상해','심하다','공감되는 건','유머요','기빨려',
    '씁쓸함','음썽','집중이','집중을','아까워','안됨','의리로','길었어','집중이 잘 안된다','잃는다','어수선하다','하차',
    '아깝','우롱','장난하나','꺼져','잼있어?','우울','산으로','아쉬움','모르겠네요','왜봤지','개판이네','재미가 없다','조잡하다',
    '몰입이 안됨','장난해?','이해안돼','초반에만','스트레스받아','정신차려라','시간 아깝다','쓸데없이','정신없어요','뭐에요;','가볍다.'
    '시작은','음','재미없어요','재미가 없다','응','없','재미없어','답답했다.','당황스러움','유치함','미흡하고','이해안돼','너무재미없다']

# 🔥 텍스트 데이터 불러오기
with open('Ontology_filtered_reviews2.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [62]:
# 🔥 데이터프레임으로 변환
dramas = []
reviews = []

for line in lines:
    line = line.strip()
    if line.startswith('[') and line.endswith(']'):
        current_drama = line.strip('[]')
    elif line != '':
        dramas.append(current_drama)
        reviews.append(line)

df = pd.DataFrame({'drama': dramas, 'review': reviews})

print(f"총 {len(df)}개의 리뷰가 있습니다.")
print(df.head())

총 982개의 리뷰가 있습니다.
                 drama                                             review
0  (아는 건 별로 없지만) 가족입니다  흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이...
1  (아는 건 별로 없지만) 가족입니다                                   가족은 같이 살아온 세월이다.
2              1%의 어떤것                                                 음.
3               18 어게인  뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지...
4               18 어게인                                     눈물이 났다 왠지 모르게…


In [63]:
# 🔥 텍스트 클렌징 함수
def clean_text(text):
    text = re.sub(r'[^가-힣\s]', '', str(text))  # 한글과 공백만 남김
    return text

def is_negative(review):
    review_clean = clean_text(review)
    tokens = okt.morphs(review_clean)  # 형태소 분석
    for kw in negative_keywords:
        if any(kw in token for token in tokens):
            return True
    return False

In [64]:
# 🔥 부정 리뷰 제거
df['is_negative'] =df['is_negative'] = df['review'].apply(
    lambda x: any(kw in clean_text(x) for kw in negative_keywords)
)
filtered_df = df[df['is_negative'] == False].drop(columns=['is_negative'])

print(f"부정 리뷰 제거 후 {len(filtered_df)}개의 리뷰가 남았습니다.")
print(filtered_df.head())

부정 리뷰 제거 후 754개의 리뷰가 남았습니다.
                 drama                                             review
0  (아는 건 별로 없지만) 가족입니다  흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이...
1  (아는 건 별로 없지만) 가족입니다                                   가족은 같이 살아온 세월이다.
4               18 어게인                                     눈물이 났다 왠지 모르게…
5               18 어게인                기분 좋아지는 휴먼 판타지 이도현의 발견, 찐 농구 장면 인정~
9                   가면                                         메이 보려고 봣어요


In [65]:
# 🔥 결과 저장
filtered_df.to_csv('Ontology_positive_reviews8.csv', index=False, encoding='utf-8-sig')
print("✅ 'filtered_reviews.csv' 파일로 저장 완료!")

✅ 'filtered_reviews.csv' 파일로 저장 완료!


In [66]:
data1 = pd.read_csv('Ontology_positive_reviews6.csv')
data2= pd.read_csv('Ontology_positive_reviews7.csv')
data3= pd.read_csv('Ontology_positive_reviews8.csv')

In [67]:
len(data1['drama'].unique())

291

In [68]:
len(data2['drama'].unique())

285

In [69]:
len(data3['drama'].unique())

279